<table>
    <tr>
        <td> 
            <img src="Slide1.png"align="center" width="250"/> 
        </td> 
        <td>
            <img src="Slide2.png"align="center"width="250"/>
        </td>
        <td>
            <img src="Slide3.png"align="center"width="250"/>
        </td>
     </tr>
</table>

## Learning objective:
* generating regional PET data
* understand the output format from the model
* learn data visualization from stoPET output

est. time 2 hour

###  Generating regional data
Here we will generate the data for **Kenya** and use some plotting functions to visualize the data. Notice regional data takes more space so make sure there is enough storage for the data.

To generate the regional data please adjust the input values in the **run_stoPET()** function and generate 5 years data for Kenya using Method 2 as temperature adjustment with a 1.5 degree increase.

### <span style='background :yellow' > Example  </span>
Now lets do a simple exercise based on a 5 year data for Kenya.

    * latval_min = -5.5
    * latval_max = 5.5
    * lonval_min = 33.0
    * lonval_max = 42.0
    * start year = 2000
    * end year = 2005
    * two ensembles
    * using Method 2 for temperature adjustment
    * temperature increase of 1.5 degrees

In [ ]:
# This is for showing plots 
# interactively in Jupiter notebook
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [ ]:
import sys
import numpy as np
from stoPET_v1 import *

In [ ]:
def run_stoPET():
    ## ----- CHANGE THE INPUT VARIABLES HERE -----##
    datapath = 'stopet_parameter_files/'
    outputpath = 'results/'
    runtype =  'regional' #'single' 
    startyear = 2000
    endyear = 2005

    # Single point stoPET run
    latval = 1.73
    lonval = 40.09

    # Regional stoPET run
    latval_min = -5.5
    latval_max = 5.5
    lonval_min = 33.0
    lonval_max = 42.0
    locname = 'Kenya'

    number_ensm = 2
    tempAdj = 2
    deltat = 1.5
    udpi_pet = 5

    ## ------ NO CHANGES BELLOW THIS -------------##
    if runtype == 'single':
            for ens_num in np.arange(0,number_ensm):
                    stoPET_wrapper_singlepoint(startyear, endyear, 
                                               latval, lonval, locname,
                                               ens_num,datapath, 
                                               outputpath, tempAdj, 
                                               deltat, udpi_pet)
    elif runtype == 'regional':
            for ens_num in np.arange(0,number_ensm):
                    stoPET_wrapper_regional(startyear, endyear, 
                                            latval_min, latval_max, 
                                            lonval_min, lonval_max,
                                            locname, ens_num, datapath, 
                                            outputpath, 
                                            tempAdj, deltat, udpi_pet)
    else:
            raise ValueError('runtype only takes single and regional ... please check!')

Now run the function run_stoPET()

In [ ]:
run_stoPET()

Lets read first year data and aggregate to annual PET. Each year’s file will have a **four dimensional array (days, hours, latitude, longitude)**. The variable name for the stochastically generated PET is **pet** within the netCDF files.

The file name of the outputs are given as **year_method_stoPET.nc** and **year_method_AdjstoPET.nc** (e.g. 2002_2_stoPET.nc, 2002_2_AdjstoPET.nc)

In [ ]:
nc = Dataset('results/Kenya_E0_StoPET/2000_2_stoPET.nc')
lats = nc.variables['latitude'][:]
lons = nc.variables['longitude'][:]
pet = nc.variables['pet'][:,:,:,:] 
# check array shape
print(pet.shape)
# make the annual sum PET
annual_pet = np.sum(pet, axis=(0,1))
# check array shape
print(annual_pet.shape)

# <font color='red'>plot_spatial function doesn't exist</font>

In [ ]:
# let us plot the annual PET
data = annual_pet
cmap = plt.cm.YlOrBr
title = 'Annual PET'
cbar_label = '$\mathbf{mm\,year^{-1}}$'
climin = 1000.0 
climax = 2500.0 
plotpath = './plots/' 
figfname = 'Kenya_annual+PET_2000.png'
plot_spatial(data, lats, lons, cmap , title, cbar_label, 
             climin, climax, plotpath, figfname)

In [ ]:
fig = plt.figure()
m = Basemap(projection='cyl', llcrnrlat=min(lats), 
            urcrnrlat=max(lats), llcrnrlon=min(lons),
            urcrnrlon=max(lons), resolution='l')

cs4 = plt.imshow(data, interpolation='nearest', cmap=cmap,
                     extent=[min(lons), max(lons), 
                             min(lats), max(lats)]) 
m.drawcoastlines(linewidth=1.0)
m.drawcountries(linewidth=0.75)
parallels=np.arange(-90.,90.,10.0)
meridians=np.arange(0.,360.,10.0)

m.drawlsmask(land_color=(0,0,0,0), ocean_color='white', lakes=False) 
plt.title(title, fontweight='bold', loc='center')
cb4 = plt.colorbar(cs4, label=cbar_label, shrink=0.4, 
                   pad=0.02, extend='both', orientation='horizontal')
cb4.mappable.set_clim(climin, climax)
plt.tight_layout()  

---
## <span style='background :red' > EXERCISE  </span>
Based on the above example on generating regional PET data please do the folowing.

    1. Generate a 5 year dataset for Uganda.
    2. Use Method 1 for temperature adjustment.
    3. Generate comparison plots side-by-side (write your own visualization code using python subplots) for the average  annual PET values of the non-adjusted PET and adjusted PET.
---   